In [19]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random


In [20]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.l1 = nn.Linear(state_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, action_dim)
        self.max_action = max_action

    def forward(self, state):
        x = torch.relu(self.l1(state))
        x = torch.relu(self.l2(x))
        x = torch.tanh(self.l3(x))
        return self.max_action * x

In [21]:
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.l1 = nn.Linear(state_dim + action_dim, 400)
        self.l2 = nn.Linear(400, 300)
        self.l3 = nn.Linear(300, 1)

    def forward(self, state, action):
        x = torch.relu(self.l1(torch.cat([state, action], 1)))
        x = torch.relu(self.l2(x))
        x = self.l3(x)
        return x


In [22]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (np.array(states), np.array(actions), np.array(rewards).reshape(-1, 1),
                np.array(next_states), np.array(dones).reshape(-1, 1))

    def size(self):
        return len(self.buffer)


In [23]:
class TD3:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-3)

        self.critic_1 = Critic(state_dim, action_dim).to(device)
        self.critic_2 = Critic(state_dim, action_dim).to(device)
        self.critic_target_1 = Critic(state_dim, action_dim).to(device)
        self.critic_target_2 = Critic(state_dim, action_dim).to(device)
        self.critic_target_1.load_state_dict(self.critic_1.state_dict())
        self.critic_target_2.load_state_dict(self.critic_2.state_dict())
        self.critic_optimizer = optim.Adam(list(self.critic_1.parameters()) + list(self.critic_2.parameters()), lr=1e-3)

        self.replay_buffer = ReplayBuffer(max_size=1_000_000)
        self.max_action = max_action
        self.discount = 0.99
        self.tau = 0.005
        self.policy_noise = 0.2
        self.noise_clip = 0.5
        self.policy_freq = 2
        self.total_it = 0

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, batch_size=100):
        self.total_it += 1

        # Sample replay buffer
        state, action, reward, next_state, done = self.replay_buffer.sample(batch_size)
        state = torch.FloatTensor(state).to(device)
        action = torch.FloatTensor(action).to(device)
        reward = torch.FloatTensor(reward).to(device)
        next_state = torch.FloatTensor(next_state).to(device)
        done = torch.FloatTensor(done).to(device)

        # Select action according to policy and add clipped noise
        noise = (torch.randn_like(action) * self.policy_noise).clamp(-self.noise_clip, self.noise_clip)
        next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)

        # Compute the target Q value
        target_q1 = self.critic_target_1(next_state, next_action)
        target_q2 = self.critic_target_2(next_state, next_action)
        target_q = reward + (1 - done) * self.discount * torch.min(target_q1, target_q2)

        # Get current Q estimates
        current_q1 = self.critic_1(state, action)
        current_q2 = self.critic_2(state, action)

        # Compute critic loss
        critic_loss = nn.MSELoss()(current_q1, target_q.detach()) + nn.MSELoss()(current_q2, target_q.detach())

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic_1(state, self.actor(state)).mean()

            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update the frozen target models
            for param, target_param in zip(self.critic_1.parameters(), self.critic_target_1.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            for param, target_param in zip(self.critic_2.parameters(), self.critic_target_2.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

    def save(self, filename):
        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(self.critic_1.state_dict(), filename + "_critic1")
        torch.save(self.critic_2.state_dict(), filename + "_critic2")

    def load(self, filename):
        self.actor.load_state_dict(torch.load(filename + "_actor.pth",map_location=device))
        self.critic_1.load_state_dict(torch.load(filename + "_critic1.pth",map_location=device))
        self.critic_2.load_state_dict(torch.load(filename + "_critic2.pth",map_location=device))


In [24]:
env = gym.make("BipedalWalker-v3", hardcore=True)
# env = gym.make("BipedalWalker-v3")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

td3 = TD3(state_dim, action_dim, max_action)

episodes = 10
batch_size = 100

In [25]:
# Load the trained actor model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [26]:
td3.load('/home/janak/Documents/Pytorch_CPU/TD3/models/Hardcore/td3_bipedal_hardcore')

In [27]:
import time

In [28]:
# To test the loaded model:
for episode in range(5):
    state = env.reset()
    episode_reward = 0
    for t in range(1600):
        env.render()
        action = td3.select_action(state)
        # print(action)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        episode_reward += reward
        if done:
            break
    print(f"Test Episode: {episode}, Reward: {episode_reward}")
    time.sleep(2)
env.close()

Test Episode: 0, Reward: 299.03219130827443
Test Episode: 1, Reward: 300.8653489879289
Test Episode: 2, Reward: 300.1543094420319
Test Episode: 3, Reward: 298.8109371130097
Test Episode: 4, Reward: 301.4381144950947
